In [2]:
import re
import pickle
import shelve
from tqdm.autonotebook import tqdm

import mwparserfromhell as mwph
from mwparserfromhell.nodes.text import Text
from mwparserfromhell.nodes.wikilink import Wikilink 
import wikitextparser as wtp

import requests
import nltk
from nltk.util import ngrams
import operator
import numpy as np
from scipy.stats import kurtosis
from Levenshtein import distance as levenshtein_distance

import time
import operator
import sys
import csv

In [3]:
# Load the anchor dictionary (the main data structure)
# this is generated by script: ./scripts/generate_anchor_dictionary.py
# enanchors = pickle.load( open( "./data/en/en.anchors.pkl", "rb" ) )
enanchors = shelve.open('./data/en/en.anchors', flag='r')

error: need 'c' or 'n' flag to open new db

In [4]:
# TODO: SVD as additional component
# We want for all pages ..

In [5]:
# Load the sentences to test
test_set = []
with open('./data/en/training/sentences_test.csv') as fin:
    for line in fin:
        try:
            title, sent = line.split('\t')
            test_set.append((title, sent))
        except:
            continue

FileNotFoundError: [Errno 2] No such file or directory: './data/en/training/sentences_test.csv'

In [6]:
# Embeddings of Wikipedia entities(not words)
# this is generated by script: wikipedia2vec train --min-entity-count=0 --dim-size 100 enwiki-latest-pages-articles.xml.bz2 ./data/en/en.w2v.bin
from wikipedia2vec import Wikipedia2Vec
w2file = './data/en/en.w2v.bin'
word2vec = Wikipedia2Vec.load(w2file)

FileNotFoundError: [Errno 2] No such file or directory: './data/en/en.w2v.bin'

In [7]:
# Navigation embeddings
# TODO: page_tile as key @Martin
# Ideal: have a vector for ALL the wikipedia pages

# TODO: Check if we can load this with mmap

import fasttext
navfile = './data/en/word2vec_enwiki_params-cbow-50-5-0.1-10-5-20.bin'
nav2vec = fasttext.load_model(navfile)

ValueError: ./data/en/word2vec_enwiki_params-cbow-50-5-0.1-10-5-20.bin cannot be opened for loading!

In [8]:
# TODO: the navigation model should change to (page_title, vector)
# This piece won't be needed then

# TODO: It is probably easier to extract the page_id from the dump
# and streamline this step

csv.field_size_limit(sys.maxsize)
reader = csv.reader(open('./data/en/pageid.csv', 'r'))
pageid = {}
for row in reader:
    k, v = row[0].split('\t')
    pageid[v] = k

FileNotFoundError: [Errno 2] No such file or directory: './data/en/pageid.csv'

In [9]:
# List of word embedded 'entities'
veclist = set([t.title for t in list(word2vec.dictionary.entities())])

NameError: name 'word2vec' is not defined

In [10]:
# Load the moodel classifier

import xgboost as xgb
model = xgb.XGBClassifier()  # init model
model.load_model('./data/en/0001.link.bin')  # load data

# make a random test of the model
# model.predict_proba(np.array([2, 36567, 669, 726.889369, 0.558646, 0.0]).reshape((1,-1)))[0,1]

XGBoostError: [05:31:03] /workspace/dmlc-core/src/io/local_filesys.cc:209: Check failed: allow_null:  LocalFileSystem::Open "./data/en/0001.link.bin": No such file or directory
Stack trace:
  [bt] (0) /home/ubuntu/pyt37/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(dmlc::io::LocalFileSystem::Open(dmlc::io::URI const&, char const*, bool)+0x506) [0x7f25b53735e6]
  [bt] (1) /home/ubuntu/pyt37/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(dmlc::Stream::Create(char const*, char const*, bool)+0x39) [0x7f25b534ab59]
  [bt] (2) /home/ubuntu/pyt37/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(XGBoosterLoadModel+0x8a3) [0x7f25b4fac1a3]
  [bt] (3) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call_unix64+0x4c) [0x7f2741d62e40]
  [bt] (4) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call+0x2eb) [0x7f2741d628ab]
  [bt] (5) /home/ubuntu/pyt37/lib/python3.7/lib-dynload/_ctypes.cpython-37m-x86_64-linux-gnu.so(_ctypes_callproc+0x2be) [0x7f2741f767fe]
  [bt] (6) /home/ubuntu/pyt37/lib/p

In [11]:
# Utility function to factor out

# Wikipedia2Vec distance between two pages (origin, destination)
# Semantic relationship
def getW2VDst(ent_a, ent_b):
    dst = 0
    if ent_a in veclist and ent_b in veclist:
        a = word2vec.get_entity_vector(ent_a)
        b = word2vec.get_entity_vector(ent_b)
        dst = (np.dot(a, b) / np.linalg.norm(a) / np.linalg.norm(b))
    return dst

In [12]:
# Utility function to factor out

# Navigation distance between two pages (origin, destination)
# Probability of navigation (see with Martin)
def getNavDst(ent_a, ent_b):
    dst = 0
    if ent_a in pageid and ent_b in pageid:
        page_a = pageid[ent_a]
        page_b = pageid[ent_b]
        if ent_a in veclist and ent_b in veclist:
            a = nav2vec.get_word_vector(page_a)
            b = nav2vec.get_word_vector(page_b)
            dst = (np.dot(a, b) / np.linalg.norm(a) / np.linalg.norm(b))
    return dst

In [13]:
# # Pre-processing

# # TODO: The following block should be replaced by finding the list of valid pages
# # to link from the Hive database


########################
# # Description: Collect a list of entities, disambiguation pages, redirects
# # to exclude them as candidates

# # The following file is from the dumps
# entities = []
# with open('./data/en_bis/enwiki-20200201-all-titles-in-ns0') as fin:
#     for line in fin:
#         entities.append(line.strip().replace("_", " "))

# # The following file is found in the redirects (I think it's available in the dumps too)
# dislinks = []
# with open('./data/en_bis/enwiki.dis') as fin:
#     for line in fin:
#         dislinks.append(line.strip())

# # Finally Remove the disambiguation links
# dislinks = set(dislinks)
# entities = set(entities)  - dislinks

In [14]:
print("Num of potential target pages:", len(veclist))

NameError: name 'veclist' is not defined

In [15]:
# Utility function to factor out

# Return the features for each link candidate in the context of the text and the page
# TODO: refactor this piece of code to be the same for training model
def get_feature_set(page, text, link):
    ngram = len(text.split()) # simple space based tokenizer to compute n-grams
    freq = enanchors[text][link] # How many times was the link use with this text 
    ambig = len(enanchors[text]) # home many different links where used with this text
    kur = kurtosis(sorted(list(enanchors[text].values()), reverse = True) + [1] * (1000 - ambig)) # Skew of usage text/link distribution
    w2v = getW2VDst(page, link) # W2V Distance between the source and target page
    nav = getNavDst(page, link) # Nav Distance between the source and target page
    return (ngram, freq, ambig, kur, w2v, nav)

In [16]:
# Main decision function.

# for a given page X and a piece of text "lipsum".. check all the candidate and make inference
# Returns the most likely candidate according to the pre-trained link model
# If the probability is below a certain threshold, return None
def classify_links(page, text, THRESHOLD):
    #start_time = time.time()
    cand_prediction = {}
    # Work with the 10 most frequent candidates
    limited_cands = enanchors[text]
    if len(limited_cands) > 10:
        limited_cands = dict(sorted(enanchors[text].items(), key = operator.itemgetter(1), reverse = True)[:10]) 
    for cand in limited_cands:
        # get the features
        cand_feats = get_feature_set(page, text, cand)
        # compute the model probability
        cand_prediction[cand] = model.predict_proba(np.array(cand_feats).reshape((1,-1)))[0,1]
    
    # Compute the top candidate
    top_candidate = max(cand_prediction.items(), key=operator.itemgetter(1))
    
    # Check if the max probability meets the threshold before returning
    if top_candidate[1] < THRESHOLD:
        return None
    #print("--- %s seconds ---" % (time.time() - start_time))
    return top_candidate

In [17]:
# Article parsing utility.

# For a given page return the list of all existing links and mentions
# To avoid linking what's already linked
def getLinks(wikicode, page_title):
    m = set()
    e = set()
    if page_title:
        page_title_tmp = page_title.replace('_',' ')
        # add the page title itself
        m.add(page_title_tmp)
        e.add(page_title_tmp)
    linklist = wtp.parse(str(wikicode)).wikilinks
    for l in linklist:
        m.add(l.plain_text().strip())
        e.add(l.title.strip())
    return m, e

In [18]:
# Article parsing utility.

# dictionary needed for evaluation

def getLinksEval(wikicode):
    link_dict={}
    linklist = wtp.parse(str(wikicode)).wikilinks
    for l in linklist:
        link_dict[l.plain_text().strip()] = l.title.strip()
    return link_dict

In [19]:
# Article parsing utility.

# Split a MWPFH node <TEXT> into sentences
SENT_ENDS = [u".", u"!", u"?"]
def tokenize_sentence_split(text):
    for line in text.split("\n"):
        tok_acc = []
        for tok in nltk.word_tokenize(line):
            tok_acc.append(tok)
            if tok in SENT_ENDS:
                yield " ".join(tok_acc)
                tok_acc = []
        if tok_acc:
            yield " ".join(tok_acc)

In [20]:
# Article parsing utility.

# This is in an effort to correct the redirects in the lists
def postProcessList(list_ent):
    correct_dict = {}
    for k in list_ent:
        if word2vec.get_entity(k):
            redirect = word2vec.get_entity(k).title
            correct_dict[redirect] = correct_dict.get(redirect, 0) + list_ent[k]
        else:
            correct_dict[k] = correct_dict.get(k, 0) + list_ent[k]
    return correct_dict

In [21]:
# Actual Linking function
def process_page(page, page_wikicode):
    page_wikicode_init= str(page_wikicode) # save the initial state
    linked_mentions, linked_links = set(), set()
    tested_mentions = set()
    for gram_length in range(10, 1, -1):
        #print("Scanning ", gram_length, "Grams")
        # Parsing the tree can be done once
        for node in page_wikicode.filter(recursive= False):
            if isinstance(node, Text):
                lines = node.split("\n")
                for line in lines:
                    for sent in tokenize_sentence_split(line):
                        grams = list(ngrams(sent.split(), gram_length))
                        for gram in grams:
                            mention = ' '.join(gram)
                            # if the mention exist in the DB 
                            # it was not previously linked (or part of a link)
                            # none of its candidate links is already used
                            # it was not tested before (for efficiency)
                            if (mention in enanchors and
                                not any(mention in s for s in linked_mentions) and
                                not bool(set(enanchors[mention].keys()) & linked_links) and
                                mention not in tested_mentions):
                                #logic
                                #print("testing:", mention, len(enanchors[mention]))
                                candidate = classify_links(page, mention, THRESHOLD)
                                if candidate:
                                    candidate_link, candidate_proba = candidate
                                    #print(">> ", mention, candidate)
                                    ############## Critical ##############
                                    # Insert The Link in the current wikitext
                                    match = re.compile(r'(?<!\[\[)(?<!-->)\b{}\b(?![\w\s]*[\]\]])'.format(re.escape(mention)))
                                    newval, found = match.subn("[[" + candidate_link  +  "|" + mention +"]]", node.value, 1)
                                    node.value = newval
                                    ######################################
                                    # Book-keeping
                                    linked_mentions.add(mention)
                                    linked_links.add(candidate)
                                # More Book-keeping
                                tested_mentions.add(mention)

    return page_wikicode

In [22]:
enanchors['unincorporated community']

NameError: name 'enanchors' is not defined

In [23]:
%%time
# Running the Model on a page

# Process pages of interest
# page_names = ["De_Lassone", "13463_Antiphos", "Peter_Jungen", "AVT"]

THRESHOLD = 0.95

### eval vars (micro and macro)
count_doc = 0.
count_docp = 0.
macro_pre = 0.
macro_rec = 0.
tot_TP = 0.
tot_rel = 0.
tot_ret = 0.

#### Backtest
for page, page_wikicode in test_set:
    input_code = page_wikicode
    output_code = process_page(page, mwph.parse(mwph.parse(page_wikicode).strip_code()))
    inp_pairs = getLinksEval(input_code)
    out_pairs = getLinksEval(output_code)
    #
    TP = dict(set(inp_pairs.items()).intersection(out_pairs.items()))
    #
    doc_pre = 0 if len(out_pairs)==0 else len(TP)/len(out_pairs)
    doc_rec = len(TP)/len(inp_pairs)
    #
    tot_TP  += len(TP)
    tot_ret += len(out_pairs)
    tot_rel += len(inp_pairs)
    #print(len(TP), len(inp_pairs), len(out_pairs), " P:", doc_pre, " R:", doc_rec)
    count_doc+=1
    if count_doc %100 == 0:
        print('----------------------')
        micro_precision = tot_TP/tot_ret
        micro_recall    = tot_TP/tot_rel
        print("micro_precision:\t", micro_precision)
        print("micro_recall:\t"   , micro_recall)

micro_precision = tot_TP/tot_ret
micro_recall    = tot_TP/tot_rel
print("micro_precision:\t",micro_precision)
print("micro_recall:\t",  micro_recall)

ZeroDivisionError: float division by zero